**Question Generator**

In [ ]:
!pip install --quiet  datasets #to access race dataset
!pip install --quiet pyarrow   #to deal with parquet files for saving dataset if required
!pip install --quiet  tqdm     #for progress bars
!pip install --quiet transformers # for t5 model
!pip install --quiet tokenizers  #tokenizers from HuggingFace
!pip install --quiet sentencepiece #subword tokenizer used by T5
!pip install --quiet pytorch-lightning # pytorch wrapper
!pip install --quiet torchtext # text utilities

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.1/731.1 kB 50.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy

In [ ]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small',model_max_length=512)
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Accesing Model

In [ ]:
trained_model_path = '/content/drive/MyDrive/t5_trained_model'
trained_tokenizer = '/content/drive/MyDrive/t5_tokenizer'
device = 'cpu'
model = T5ForConditionalGeneration.from_pretrained(trained_model_path)
tokenizer = T5Tokenizer.from_pretrained(trained_tokenizer)

In [ ]:
trained_model_path1 = '/content/drive/MyDrive/t5_trained_model2'
trained_tokenizer1 = '/content/drive/MyDrive/t5_tokenizer2'
device = 'cpu'
model1 = T5ForConditionalGeneration.from_pretrained(trained_model_path1)
tokenizer1 = T5Tokenizer.from_pretrained(trained_tokenizer1)

In [ ]:
def get_question1(sentence,mdl,tknizer):

  ''' function to generate questions. Takes a sentence,answer,
      model and tokenizer
  '''

  text = "context: {}".format(sentence)
  #print (text)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=72)
  #print("outputs")
  #print(outs)
  for beam_output in outs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    #print(sent)

  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]
  #print("decs")
  #print(dec)

  Question = dec[0]
  #Question= Question.strip()
  #Answer= Answer.strip()
  #print(Answer)
  index = Question.find("answer:")

  # Extract the question and answer based on the position
  question = Question[10:index].strip()
  answer = Question[index + len("answer:"):].strip()
  #print("Question1:", question)
  #print("Answer1:", answer)

  return  question,answer

In [ ]:
def divide_into_paragraphs(text, num_paragraphs):
    # Split the text into paragraphs based on the desired number
    words = text.split()
    words_per_paragraph = len(words) // num_paragraphs

    paragraphs = []
    start_index = 0
    for _ in range(num_paragraphs - 1):
        end_index = start_index + words_per_paragraph
        paragraph = " ".join(words[start_index:end_index])
        paragraphs.append(paragraph)
        start_index = end_index

    # Add the remaining words as the last paragraph
    last_paragraph = " ".join(words[start_index:])
    paragraphs.append(last_paragraph)

    return paragraphs

Integrating Bert Similarity

In [ ]:
class STSBertModel(torch.nn.Module):

    def __init__(self):

        super(STSBertModel, self).__init__()

        word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=128)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
        self.sts_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    def forward(self, input_data):

        output = self.sts_model(input_data)

        return output

Accessing SBERT

In [ ]:
!pip install sentence_transformers
import joblib
trained_model = joblib.load('/content/drive/MyDrive/Copy of Sbert_Model_1')
!pip install datasets
!pip install sentence-transformers
!pip install transformers
!pip install torchmetrics
import torch
from sentence_transformers import SentenceTransformer, models
from transformers import BertTokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
from datasets import load_dataset
import matplotlib.pyplot as plt
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=a7a2b7f5bf747fa2f0574cf0878ac41577d74843d0e7d5f99b642e33298eafa6
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
# Function to predict test data
def predict_sts(texts):

  trained_model .to('cpu')
  trained_model.eval()
  test_input = tokenizer(texts, padding='max_length', max_length = 128, truncation=True, return_tensors="pt")
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']

  test_output = trained_model(test_input)['sentence_embedding']
  sim = torch.nn.functional.cosine_similarity(test_output[0], test_output[1], dim=0).item()

  return sim

In [ ]:
def process_question(question,questions):
  similarity = False
  compare_questions = []
  similar_question = " "
  for value in questions:
    if (similarity == False):
      compare_questions.append(question)
      #print('Q',question)
      compare_questions.append(value)
      #print('V',value)
      if predict_sts(compare_questions) > 0.8 :
        similarity = True
        similar_question = value
      compare_questions.clear()
  return similar_question


In [ ]:
def process_answer(answer,answers):
  similar_answer = False
  compare_answers = []
  for value in answers:
    if (similar_answer == False):
      compare_answers.append(answer)
      compare_answers.append(value)
      if predict_sts(compare_answers) > 0.8:
        similar_answer = True
      compare_answers.clear()
  return similar_answer

Final Output from Model

In [ ]:
!pip install PyPDF2
import PyPDF2
import math
import re

def qa_func(start_page: int, end_page: int, num_qa: int):
  #qa_dict=dict.fromkeys('Question','Answer')
  qa_dict={}
  figure_regex = r"\b(Fig(?:ure)?\.?\s\d+)\b"
  table_regex = r"\b(Table\s\d+)\b"
  num_qa_per_pg = math.ceil(num_qa/(end_page-start_page+1))
  if (num_qa_per_pg>5): #5 is hyperparamer which means it will generate 5 questions per page
    print('Enter less number of questions')
  else:

      #pdf_file_path = "/content/drive/MyDrive/Machine Learning - Tom Mitchell_compressed.pdf"
      pdf_file_path ="/content/pdf2generate.pdf"
      with open(pdf_file_path, "rb") as file:
          pdf_reader = PyPDF2.PdfReader(file)
          #pdf_reader = PyPDF2.PdfReader(pdf_file_path)

          # Find the start and end page numbers of the desired chapter from the index
          # start_page = 10  # Replace with the actual start page number of the chapter
          # end_page = 20    # Replace with the actual end page number of the chapter

          # Extract pages of the desired chapter
          extracted_pages = []
          count=0
          for page_num in range(start_page, end_page+1):
              page = pdf_reader.pages[page_num].extract_text()
              #print(page_num)

              # Remove figures, tables, author names, and titles

              page = re.sub(figure_regex, "", page, flags=re.IGNORECASE)

              page = re.sub(table_regex, "", page, flags=re.IGNORECASE)

              paragraphs = divide_into_paragraphs(page, num_qa_per_pg)
              for i, paragraph in enumerate(paragraphs):
                #print(i)

                if (count<num_qa):
                  #print(count)
                  ques,answer = get_question1(paragraph,model1,tokenizer1)
                  #print(ques)
                  #if(len(qa_dict) > 0):
                  if len(qa_dict) == 0 :
                    qa_dict[ques] = [answer]
                    count=count+1
                  if count > 0:
                    similar_question = process_question(ques,qa_dict.keys())
                    if similar_question == " " :
                      qa_dict[ques] = [answer]
                      count=count+1
                    else:
                      answer_list = qa_dict[similar_question]
                      similar_answer = process_answer(answer, answer_list)
                      if similar_answer == False :
                        answer_list.append(answer)
                        qa_dict[similar_question] = answer_list
                else:
                  break
  return qa_dict
#my_dict={}
#my_dict=qa_func(15,16,7)
#print(my_dict)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [ ]:
'''x = my_dict.keys()
print(len(x))
for i in x:
  print(i)'''

'x = my_dict.keys()\nprint(len(x))\nfor i in x:\n  print(i)'

**Student's Answer Evaluation**

In [ ]:
'''z = my_dict.values()
print(len(z))
for i in (z):
  print(i)'''

'z = my_dict.values()\nprint(len(z))\nfor i in (z):\n  print(i)'

In [ ]:
'''compare_answers = []
for i,j in zip(my_dict.values(),my_dict.keys()):
  print('The question is: ',j)
  ansS = input("\nPlease enter the corresponding answer from the student: ")
  compare_answers.append(i)
  compare_answers.append(ansS)
  #print(compare_answers)
  print('This similarity to the original answer is: ',predict_sts(compare_answers))
  compare_answers.clear()'''

'compare_answers = []\nfor i,j in zip(my_dict.values(),my_dict.keys()):\n  print(\'The question is: \',j)\n  ansS = input("\nPlease enter the corresponding answer from the student: ")\n  compare_answers.append(i)\n  compare_answers.append(ansS)\n  #print(compare_answers)\n  print(\'This similarity to the original answer is: \',predict_sts(compare_answers))\n  compare_answers.clear()'

In [ ]:
!pip install anvil-uplink
import anvil.server
import base64
import os
anvil.server.connect("server_E3FCTYIOKPCC246OZC7ZI3IS-NOPF6GOREISCK3NF")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45227 sha256=679ed24740e8c5f160ea275c73da301617fb84354a6a470561995c95384aaee0
  Stored in directory: /root/.cache/pip/wheels/2e/7c/ad/d9c746276bf024d44296340869fcb169f1e5d80fb147351a57
Successfully built ws4py


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
@anvil.server.callable
def value_check(file_data, N, s, e):
  if os.path.exists("pdf2generate.pdf"):
        os.remove("pdf2generate.pdf")
  file_bytes = base64.b64decode(file_data)
  with open("pdf2generate.pdf", "wb") as f:
    f.write(file_bytes)
  from google.colab import files
  files.download('pdf2generate.pdf')
  print(int(N))
  print(int(s))
  print(int(e))
  my_dict={}
  my_dict=qa_func(s,e,N)
  questions = list(my_dict.keys())
  return questions, my_dict

In [ ]:
@anvil.server.callable
def user_answer_check(userQnA, my_dict):
  questions = list(userQnA.keys())
  score_dict = {}
  for question in questions:
    score = 0
    compare_answers = []
    userAnswer = userQnA[question]
    answer_list = my_dict[question]
    for answer in answer_list:
      compare_answers.append(userAnswer)
      compare_answers.append(answer)
      new_score = predict_sts(compare_answers)
      if new_score > score :
        score = new_score
      compare_answers.clear()
    if score < 0.6 :
      score = 0
    score_dict[question] = round(score,3)
  return score_dict

In [ ]:
anvil.server.wait_forever()